# PSB sequence

This includes the correct ramp rate and SET feedback as well.

In [18]:
%matplotlib inline
import local_broom

In [19]:
from zhinst.toolkit import Session, CommandTable, Sequence, Waveforms, SHFQAChannelMode

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

import time
from monty import Monty

In [20]:
# Functions

In [21]:
def timeToSamples(time, samplingRateDivider):
    # returns the number of samples divisible by 16 given a time (in seconds) and sampling rate divider
    samples_raw = time * (1/(2**samplingRateDivider))/0.5e-9
    samples_modulo = int(samples_raw) % 16
    samples = int(samples_raw) - int(samples_modulo)
    return samples

# custom curve fit
amps = [1, 0.85, 0.75, 0.65,  0.5, 0.35, 0.25, 0.2, 0.125, 0.05]
voltages = [0.34, 0.268, 0.207, 0.158, 0.094, 0.0481,  0.026, 0.0183, 0.01, 0.0057]

alt_fit = np.poly1d(np.polyfit(amps, voltages, 2))
# Hard coding alt_fit values in case something goes horribly wrong

def voltToDbm(volt, dbmrange):
    # Ok yes this can be better, deal with it
    if dbmrange != 0:
        raise Exception("This function only works with a dBm range of 0.")
    
    if volt > 0.34 or volt < -0.34:
        raise ValueError(f"Given voltage ({volt} V) is greater than max output of SHFQC (0.34 V)")
    
    if volt < 0:
        amplitude = 1/300*(np.sqrt(3e5*-volt + 529) - 23)
        return -amplitude
    else:
        amplitude = 1/300*(np.sqrt(3e5*volt + 529) - 23)
        return amplitude


def get_results(result_node, timeout):
    wave_data_captured = {}
    wave_data_captured[result_node] = False
    start_time = time.time()
    captured_data = {}
    while not all(wave_data_captured.values()):
        if start_time + timeout < time.time():
            print(captured_data)
            raise TimeoutError('Timeout before all samples collected.')
        test = session.poll()
        for node, value in test.items():
            node = session.raw_path_to_node(node)
            for v in value:
                if node not in captured_data:
                    captured_data[node] = [v['vector']]
                else:
                    captured_data[node].append(v['vector'])
            if len(captured_data[node]) >= 1:  # readout 1 point
                wave_data_captured[node] = True
                # total_num_data = sum([len(element) for element in captured_data[node]])
    data = captured_data[result_node][0]
    return data

def cmdtable(ct, amplitude, length, wave_index, ct_index):
    """
    Load a default command table with a sin/cos wave (used throughout the documentation)
    """
    ct.table[ct_index].waveform.index = wave_index
    ct.table[ct_index].amplitude00.value = amplitude  # all in dBm
    ct.table[ct_index].amplitude01.value = -amplitude
    ct.table[ct_index].amplitude10.value = amplitude
    ct.table[ct_index].amplitude11.value = amplitude
    ct.table[ct_index].waveform.length = length  # in samples
    ct.table[ct_index].waveform.samplingRateDivider = samplingDivider  # inherit global

# Connect

In [22]:
# connect to instruments

DEVICE_ID = 'DEV12158'
SERVER_HOST = 'localhost'

# connect to data server
session = Session(SERVER_HOST)

# connect to device
device = session.connect_device(DEVICE_ID)

In [23]:
#session.disconnect_device(DEVICE_ID)

In [24]:
experiment = {
    "desc": "PSB sequence."
}

monty = Monty("rf.psb", experiment)

Started new experiment rf.psb


In [25]:

# lengths and averages
averages = 1  # python averages
seq_averages = 1000  # seqc averages

NUM_DETUNING = 100
NUM_J = 1

# readout signals
readout_gain = 0.95  # If we set this to 1, then output overloads
readout_freq = 406.2e6  # (Hz)

# drive line amplitudes (maximum 340mV, give in terms of V)
# pulse parameters
MIXED_AMP = {  # voltage amplitude for mixed state preperation
    "P1": 0.1,
    "P2": 0.1,
}
MES_AMP_START = {  # voltages for stepping over when sweeping
    "P1": 0.1,
    "P2": 0.2,
    "J": 0.0,
}
MES_AMP_END = {
    "P1": 0.2,
    "P2": 0.1,
    "J": 0.0,
}


detuning_list = np.linspace(0, 0.1, NUM_DETUNING)  # for p1, take the negative of this list
j_list = np.linspace(-0.3, 0, NUM_J)

# timings in seconds
init_len = 600e-6
read_len = 100e-6
wait_and_settle = 3e-3
trigger_time = 10e-3  # internal trigger holdoff time (metronome)

# AC decay ramp rate
ramp_rate = 13e-5  # V/sample

# powers
input_pwr = -0
output_pwr = -0
dr_pwr = 0

# gate sampling time
samplingDivider = 6

# feedback parameters
FB_TARGET = 1e-5  # give in terms of power output
FB_STEPSIZE = 1e-4
FB_SLOPE = 1  # 1 or -1 depending on the slope to lock onto

# internal
read_samples = timeToSamples(read_len, samplingDivider)


In [26]:
# adjust variables
curr_st_amp = 0.3 # NEED TO FIX THIS...

# Create channel maps for simplicity

chan = {
    "measure": device.qachannels[0],  # measure and acquire lines
    "ST": device.sgchannels[0],
    "P1": device.sgchannels[1],  # drive P1 line
    "P2": device.sgchannels[2],  # drive P2 line
    "J": device.sgchannels[3],  # drive J line
}

drive_chans = ["ST", "P1", "P2", "J"]  # match keys above

# current command tables
cts = {c: CommandTable(chan[c].awg.commandtable.load_validation_schema()) for c in drive_chans}

j_steps = np.linspace(MES_AMP_START["J"], MES_AMP_END["J"], NUM_J)
p1_steps = np.linspace(MES_AMP_START["P1"], MES_AMP_END["P1"], NUM_DETUNING)
p2_steps = np.linspace(MES_AMP_START["P2"], MES_AMP_END["P2"], NUM_DETUNING)

In [27]:
# check we can fit all the points into memory
MAX_MEASUREMENTS = 2**19
if seq_averages > MAX_MEASUREMENTS:
    raise OverflowError("Requested too many points to be measured. (Use around 500,000 points)")

In [28]:
params = {
    "todo": True

}

# Channels

In [29]:
with device.set_transaction():
    # setup drive channels
    for c in drive_chans:
        chan[c].output.range(dr_pwr)  # in dBm
        chan[c].output.rflfpath(0)  # use LF not RF (1 for RF)
    
        # set the center synth frequency (oscillator frequency)
        synth = chan[c].synthesizer()
        device.synthesizers[synth].centerfreq(0)  # in Hz
        chan[c].output.on(1)  # enable output
    
        chan[c].awg.outputamplitude(1.0)  # overall amplitude scaling factor (don't really need to change)
        chan[c].oscs[0].freq(0)  # oscillator 1 frequency (Hz) disable for DC
        chan[c].oscs[1].freq(0)  # oscillator 2 frequency (Hz)
        chan[c].awg.modulation.enable(1)  # start digital modulation
    
        chan[c].marker.source(0)  # setup the AWG trigger 1 (is this an input trigger option? doesn't seem necessary)
        # see manual page p235 for all trigger options
        chan[c].awg.auxtriggers[0].channel(8)  # 8=use internal trigger, 1024=use software trigger

    # setup measure channel
    
    chan["measure"].output.rflfpath(0)  # use LF mode not RF (for signals under 600Mhz)
    chan["measure"].input.rflfpath(0)
    chan["measure"].oscs[0].freq(readout_freq)  # CW frequency (in LF mode)
    chan["measure"].oscs[0].gain(readout_gain)  # If we set this to 1, then output overloads

    # configure these based on how the sweeper works internally
    # See https://docs.zhinst.com/zhinst-utils/en/latest/_modules/zhinst/utils/shf_sweeper.html#ShfSweeper
    chan["measure"].spectroscopy.delay(0)  # integration delay in units of second
    chan["measure"].spectroscopy.length(timeToSamples(read_len, samplingDivider))  # integration time length in units of number of samples (usually integration_time*sampling_rate)
    # setup when the spectroscopy is triggered
    chan["measure"].spectroscopy.trigger.channel("chan0seqtrig0")  # make sure to use the trigger coming from the sequencer code
    # setup result parameters
    chan["measure"].spectroscopy.result.averages(1)  # number of averages (always average in software not hardware)
    chan["measure"].spectroscopy.result.length(seq_averages*2)  # number of results
    chan["measure"].spectroscopy.result.enable(0)  # disable result logger

    chan["measure"].configure_channel(
        center_frequency=0,  # in units of Hz  # minimum of 600MHz for RF mode
        input_range=input_pwr,  # in units of dBm
        output_range=output_pwr,  # in units of dBm
        mode=SHFQAChannelMode.SPECTROSCOPY,  # SHFQAChannelMode.READOUT or SHFQAChannelMode.SPECTROSCOPY
    )
    
    chan["measure"].input.on(1)
    chan["measure"].output.on(1)

    chan["measure"].generator.auxtriggers[1].channel("inttrig")  # i believe this is overwritten by the following line
    chan["measure"].generator.configure_sequencer_triggering(
        aux_trigger=8,  # alternatively use 8=internal trigger, or "software_trigger0" to use the software triggering system
        play_pulse_delay=0
    )


# Ramped pulses

In [30]:
# # define waveforms as list of real-values arrays

# waves_p1 = []
# waves_p2 = []
# for i, val in enumerate(detuning_list):

#     wv_p1 = np.linspace(voltToDbm(val, 0), voltToDbm(val+val*read_samples*ramp_rate, 0), read_samples)
#     wv_p2 = np.linspace(voltToDbm(-val, 0), voltToDbm(-val-val*read_samples*ramp_rate, 0), read_samples)

#     waves_p1.append(wv_p1)
#     waves_p2.append(wv_p2)


# Sequence

*P1/P2 sequence explanation*: The sequence starts at a "home" point with 0 V on the P1/P2/J pulsing lines. After waiting for a trigger, the sequence will immediately go to it's initialisation point (executeTableEntry(0)) and wait for a set period of time (here: wait_and_settle, read_len). After a J pulse between wait_and_settle and read_len events P1 and P2 will be pulsed to an arbitrary point (executeTableEntry(1)) which is uploaded and updated every time in python after SET feedback has occured.

In [31]:
seqc_program_p1 = f"""
// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""

seqc_program_p2 = f"""
// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""

seqc_program_j = f"""
// Assign a single channel waveform to wave table entry 0
wave w_j = ones({timeToSamples(init_len, samplingDivider)});
assignWaveIndex(1,2, w_j, 0);

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    playZero({timeToSamples(init_len, samplingDivider)},  {samplingDivider});           // mixed state init.
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle

    executeTableEntry(1);                                                               // read reference
    
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""



seqc_program_st = f"""
// Assign a single channel waveform to wave table entry 0
wave w_st = ones({timeToSamples(trigger_time, 9)});
assignWaveIndex(1,2, w_st, 0);

repeat({seq_averages}) {{
    executeTableEntry(0);
}}
"""

In [32]:
seqc_program_prior_read = f"""
repeat({seq_averages}) {{
    waitDigTrigger(1);
    
    setTrigger(1);
    setTrigger(0);
}}
"""

readout_prog_code = f"""
setTrigger(0); // Set low as this starts the spectroscopy readout....

repeat({seq_averages}) {{
    waitDigTrigger(1);
    
    playZero(224); // lineup with SG trigger (224 samples = lines up with SG trigger);
    
    playZero({timeToSamples(init_len, samplingDivider)},  {samplingDivider});
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});
    playZero(32);

    setTrigger(1);  // trigger the output. As this matches "chan0seqtrig0" the spectroscopy is started
    setTrigger(0);

    playZero({timeToSamples(read_len*2, samplingDivider)},  {samplingDivider});
    playZero(32);

    setTrigger(1);  // trigger the output. As this matches "chan0seqtrig0" the spectroscopy is started
    setTrigger(0);
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider}); 
    }}

"""

In [33]:
# create waveforms
samples = timeToSamples(read_len, samplingDivider)
default_amp = 0.01

mixed_p1 = np.linspace(voltToDbm(default_amp, dr_pwr),
                       voltToDbm(default_amp+default_amp*samples*ramp_rate, dr_pwr), samples)
mixed_p2 = np.linspace(voltToDbm(default_amp, dr_pwr),
                       voltToDbm(default_amp+default_amp*samples*ramp_rate, dr_pwr), samples)

In [34]:
# upload drive code

# P1
seq = Sequence()
seq.code = seqc_program_p1
seq.waveforms = Waveforms()
seq.waveforms[0] = mixed_p1
chan["P1"].awg.load_sequencer_program(seq)
chan["P1"].awg.write_to_waveform_memory(seq.waveforms)
print(f"_________ {chan["P1"]} _________")
print(seq.code)

# P2
seq = Sequence()
seq.code = seqc_program_p2
seq.waveforms = Waveforms()
seq.waveforms[0] = mixed_p2
chan["P2"].awg.load_sequencer_program(seq)
chan["P2"].awg.write_to_waveform_memory(seq.waveforms)
print(f"_________ {chan["P2"]} _________")
print(seq.code)

# J
chan["J"].awg.load_sequencer_program(seqc_program_j)
print(f"_________ {chan["J"]} _________")
print(seqc_program_j)

# ST
chan["ST"].awg.load_sequencer_program(seqc_program_st)
print(f"_________ {chan["ST"]} _________")
print(seqc_program_st)

# upload wavevforms with sequence code as the memory must be allocated first

# upload measure code
#chan["measure"].generator.load_sequencer_program(readout_prog_code)
#chan["measure"].generator.load_sequencer_program(seqc_program_prior_read)

_________ /dev12158/sgchannels/1 _________

// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat(1000) {

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero(93744,  6);    // wait and settle
    playZero(3120,  6);           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero(93744,  6);    // wait and settle
}

_________ /dev12158/sgchannels/2 _________

// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat(1000) {

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init

# Command tables

In [35]:
# load mixed state command channels
for c in ["P1", "P2"]:
    cmdtable(cts[c],
             amplitude= voltToDbm(MIXED_AMP[c], chan[c].output.range()),
             length=timeToSamples(init_len, samplingDivider),
             wave_index=0,
             ct_index=0,
            )

In [36]:
def uploadcts():
    """Upload the command tables to the device"""
    for c in drive_chans:
        chan[c].awg.commandtable.upload_to_device(cts[c])

In [37]:
def waitForInternalTrigger():
    """
    Waits for the internal trigger to finish running and shows the current progress.
    """
    pbar = tqdm(total=100)
    while device.system.internaltrigger.progress() != 1.0:
        p = int(device.system.internaltrigger.progress()*100)
        pbar.update(p-pbar.n)
        time.sleep(0.001)
    pbar.update(100-pbar.n)
    pbar.close()

In [38]:
def calculatefeedback(data):
    """Calculate the appropriate feedback adjustment to ST."""
    # use last datapoint
    global curr_st_amp
    error = FB_TARGET-np.abs(data[-1])
    curr_st_amp += error*FB_STEPSIZE*FB_SLOPE

    # update command table
    cmdtable(cts["ST"],
             amplitude= voltToDbm(curr_st_amp, chan["ST"].output.range()),
             length=timeToSamples(trigger_time, 9),
             wave_index=0,
             ct_index=0,
            )
    

In [39]:
def movemeasurement(p1, p2, j):
    """Move P1/P2/J to measure the next appropriate datapoint."""
    cmdtable(cts["P1"],
             amplitude= voltToDbm(p1, chan["P1"].output.range()),
             length=timeToSamples(read_len, samplingDivider),
             wave_index=0,
             ct_index=1,
            )
    cmdtable(cts["P2"],
             amplitude= voltToDbm(p2, chan["P2"].output.range()),
             length=timeToSamples(read_len, samplingDivider),
             wave_index=0,
             ct_index=1,
            )
    cmdtable(cts["J"],
             amplitude= voltToDbm(j, chan["J"].output.range()),
             length=timeToSamples(read_len, samplingDivider),
             wave_index=0,
             ct_index=1,
            )
    

In [40]:
def runfeedback():
    """Run feedback measurement where we only measure"""
    device.system.internaltrigger.enable(0)

    result_node = chan["measure"].spectroscopy.result.data.wave
    result_node.subscribe()
    
    chan["measure"].spectroscopy.result.enable(1)  # start logger
    chan["measure"].generator.enable_sequencer(single=True)
    device.system.internaltrigger.enable(1)
    
    time.sleep(0.2)  # delay for networking issues
        
    # wait for the measurement to complete
    waitForInternalTrigger()
    
    m_state = chan["measure"].generator.sequencer.status()
    if m_state != 4:
        print(TimeoutError(f"Sequencers in unknown state. Perhaps they are not synchronised? {bin(m_state)}"))
        time.sleep(0.5)
    
    # wait for completion
    while chan["measure"].spectroscopy.result.enable() != 0:
        print(chan["measure"].spectroscopy.result.enable())
        chan["measure"].spectroscopy.result.enable.wait_for_state_change(0, timeout=10)
    
    # get results
    results = get_results(result_node, timeout=5)
    result_node.unsubscribe()
    
    # verify results
    acq = chan["measure"].spectroscopy.result.acquired()
    if len(results) > acq:
        print(chan["measure"].generator.ready())
        raise TimeoutError(f"Not all datapoints measured in the time provided. {acq} of {len(results)}.")

    return np.mean(results)
    

In [41]:
def runexperiment():
    device.system.internaltrigger.enable(0)

    result_node = chan["measure"].spectroscopy.result.data.wave
    result_node.subscribe()
    
    chan["measure"].spectroscopy.result.enable(1)  # start logger
    
    
    # start sequencers
    chan["measure"].generator.enable_sequencer(single=True)
    chan["J"].awg.enable_sequencer(single=True)  # dont want to repeat
    chan["P1"].awg.enable_sequencer(single=True)
    chan["P2"].awg.enable_sequencer(single=True)
    chan["ST"].awg.enable_sequencer(single=True)
    
    
    # start triggering sequence (which starts each sequencer)
    device.system.internaltrigger.enable(1)
    time.sleep(0.2)  # delay for networking issues
    
        
    # wait for the measurement to complete
    waitForInternalTrigger()
    
    
    # check sequencers have finished their sequences
    # Status of the Sequencer on the instrument.
    # - Bit 0: Sequencer is running;
    # - Bit 1: reserved;
    # - Bit 2: Sequencer is waiting for a trigger to arrive;
    # - Bit 3: Sequencer has detected an error;
    # - Bit 4: sequencer is waiting for synchronization with other channels
    m_state = chan["measure"].generator.sequencer.status()
    st_state = chan["P1"].awg.sequencer.status()
    if m_state != 4 and st_state != 4:
        print(TimeoutError(f"Sequencers in unknown state. Perhaps they are not synchronised? {bin(m_state)}, {bin(st_state)}"))
        time.sleep(0.5)
    
    
    # wait for completion
    while chan["measure"].spectroscopy.result.enable() != 0:
        print(chan["measure"].spectroscopy.result.enable())
        chan["measure"].spectroscopy.result.enable.wait_for_state_change(0, timeout=10)
    
    
    # get results
    results = get_results(result_node, timeout=5)
    result_node.unsubscribe()
    
    
    # verify results
    acq = chan["measure"].spectroscopy.result.acquired()
    if len(results) > acq:
        print(chan["measure"].generator.ready())
        print([chan[c].awg.ready() for c in drive_chans])
        raise TimeoutError(f"Not all datapoints measured in the time provided. {acq} of {len(results)}.")
    
    return np.mean(results.reshape((seq_averages, 2)), axis=0)




In [42]:
device.system.internaltrigger.repetitions(seq_averages)  # make sure that this matches how many pulses we are sending
device.system.internaltrigger.holdoff(trigger_time)  # init_len + 2*wait_and_settle + 100e-6*2)  # how long to wait between retriggers (increments of 100ns)

In [43]:
# setup channel synchronization
chan["measure"].synchronization.enable(1)
chan["P1"].synchronization.enable(1)
chan["P2"].synchronization.enable(1)
chan["J"].synchronization.enable(1)
chan["ST"].synchronization.enable(1)
device.system.internaltrigger.synchronization.enable(1)

# Run

In [44]:
#monty.newrun("feedback", params)

# work out what we should be doing for feedback
with device.set_transaction():
    chan["measure"].synchronization.enable(1)
    chan["P1"].synchronization.enable(0)
    chan["P2"].synchronization.enable(0)
    chan["J"].synchronization.enable(0)
    chan["ST"].synchronization.enable(0)
    device.system.internaltrigger.synchronization.enable(1)
    
    chan["measure"].spectroscopy.result.length(seq_averages)
    chan["measure"].generator.load_sequencer_program(seqc_program_prior_read)
time.sleep(1)

data = [runfeedback()]

with device.set_transaction():
    chan["measure"].synchronization.enable(1)
    chan["P1"].synchronization.enable(1)
    chan["P2"].synchronization.enable(1)
    chan["J"].synchronization.enable(1)
    chan["ST"].synchronization.enable(1)
    device.system.internaltrigger.synchronization.enable(1)
    
    chan["measure"].generator.load_sequencer_program(readout_prog_code)
    chan["measure"].spectroscopy.result.length(seq_averages*2)
time.sleep(1)

for j in j_steps:
    for (p1, p2) in zip(p1_steps, p2_steps):
        movemeasurement(p1, p2, j)  # CT = 1
        calculatefeedback(data)  # CT = 0
        uploadcts()
        data = runexperiment()
        

  0%|          | 0/100 [00:00<?, ?it/s]

Sequencers in unknown state. Perhaps they are not synchronised? 0b0


  0%|          | 0/100 [00:00<?, ?it/s]

Sequencers in unknown state. Perhaps they are not synchronised? 0b0, 0b0


  0%|          | 0/100 [00:00<?, ?it/s]

...

Sequencers in unknown state. Perhaps they are not synchronised? 0b0, 0b0


  0%|          | 0/100 [00:00<?, ?it/s]

Sequencers in unknown state. Perhaps they are not synchronised? 0b0, 0b0


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.plot(np.linspace(0, pulse_time, read_lens), np.abs(data))  
plt.ylabel("RF signal (arb units)")
plt.title(monty.identifier + "." + monty.runname)

monty.save({"data": data})
monty.savefig(plt, "ramp decay")